In [253]:
from __future__ import print_function
import tweepy
import json
from pymongo import MongoClient

In [254]:
# cursor = db.collection.find().limit(200).sort([("created_at",-1)])

In [255]:
MONGO_HOST= 'mongodb://127.0.0.1:27017/twitterdb'
client = MongoClient(MONGO_HOST)
            
            # Use twitterdb database. If it doesn't exist, it will be created.
db = client.twitterdb

cursor = db.twitter_search.find().limit(10000).sort([("created_at",-1)])

import pandas as pd
new_df = pd.DataFrame() #columns = ['location']
HASHTAGS = ['airpod','ipad','macbook','iphone','iwatch']

states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
for post in cursor:
    if str(post['user']['location']).split(', ')[-1] in states:
        #print('loc', str(post['user']['location']).split(', ')[-1])
        for i, hashtag in enumerate(post['entities']['hashtags']):
            #print(post['entities']['hashtags'][i]['text'])
            if post['entities']['hashtags'][i]['text'] in HASHTAGS:
                print(str(post['source']))
                device = ''
                if 'Android' in str(post['source']):
                    device = 'Andriod'
                    print('Andriod')
                elif 'Web' in str(post['source']):
                    device = 'Web'
                    print('Web')
                elif 'iPhone' in str(post['source']):
                    device = 'iPhone'
                    print('iPhone')
                elif 'Buffer' in str(post['source']):
                    device = 'Buffer'
                    print('Buffer')
                elif 'None' in str(post['source']):
                    device = 'None'
                    print('None')
                else:
                    continue
                new_df = new_df.append({'location' : str(post['user']['location']).split(', ')[-1], 
                                        'time' : post['created_at'],
                                        'tag' : post['entities']['hashtags'][i]['text'],
                                        'device' : device},
                                         ignore_index=True)
    



<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
Andriod
<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
Andriod
<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
Andriod
<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
Andriod
<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
Andriod
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>
Web
<a href="https://buffer.com" rel="nofollow">Buffer</a>
Buffer


In [256]:
new_df.describe()

,device,location,tag,time
count,7,7,7,7
unique,3,3,2,7
top,Andriod,FL,ipad,Fri Nov 30 05:44:03 +0000 2018
freq,5,5,5,1


In [244]:
new_df.head(10)

,device,location,tag,time
0,Andriod,FL,ipad,Fri Nov 30 05:44:03 +0000 2018
1,Andriod,FL,ipad,Fri Nov 30 05:40:01 +0000 2018
2,Andriod,FL,ipad,Fri Nov 30 05:35:58 +0000 2018
3,Andriod,FL,ipad,Fri Nov 30 05:31:56 +0000 2018
4,Andriod,FL,ipad,Fri Nov 30 05:26:55 +0000 2018
5,Web,TX,iphone,Fri Nov 30 05:18:49 +0000 2018
6,Buffer,AZ,iphone,Fri Nov 30 05:08:02 +0000 2018


In [209]:
new_df['time']

0    Fri Nov 30 05:44:03 +0000 2018
1    Fri Nov 30 05:40:01 +0000 2018
2    Fri Nov 30 05:35:58 +0000 2018
3    Fri Nov 30 05:31:56 +0000 2018
4    Fri Nov 30 05:26:55 +0000 2018
5    Fri Nov 30 05:18:49 +0000 2018
6    Fri Nov 30 05:08:02 +0000 2018
Name: time, dtype: object

In [210]:
for time in enumerate(new_df['time']):
    time_meta = str(time[1]).split(' ')
    print(time_meta)

['Fri', 'Nov', '30', '05:44:03', '+0000', '2018']
['Fri', 'Nov', '30', '05:40:01', '+0000', '2018']
['Fri', 'Nov', '30', '05:35:58', '+0000', '2018']
['Fri', 'Nov', '30', '05:31:56', '+0000', '2018']
['Fri', 'Nov', '30', '05:26:55', '+0000', '2018']
['Fri', 'Nov', '30', '05:18:49', '+0000', '2018']
['Fri', 'Nov', '30', '05:08:02', '+0000', '2018']


In [211]:
a = new_df['time'].apply(lambda x: str(x).split(' '))
a = a.apply(lambda x:x[2]+'-'+x[1]+'-'+x[5] + ' ' + x[3])
new_df['time'] = a.apply(lambda x:datetime.datetime.strptime(x,'%d-%b-%Y %H:%M:%S').strftime('%Y-%m-%d %H:%M:%S') )
print(a)

0    30-Nov-2018 05:44:03
1    30-Nov-2018 05:40:01
2    30-Nov-2018 05:35:58
3    30-Nov-2018 05:31:56
4    30-Nov-2018 05:26:55
5    30-Nov-2018 05:18:49
6    30-Nov-2018 05:08:02
Name: time, dtype: object


In [212]:
new_df.head(10)

,device,location,tag,time
0,Andriod,FL,ipad,2018-11-30 05:44:03
1,Andriod,FL,ipad,2018-11-30 05:40:01
2,Andriod,FL,ipad,2018-11-30 05:35:58
3,Andriod,FL,ipad,2018-11-30 05:31:56
4,Andriod,FL,ipad,2018-11-30 05:26:55
5,Web,TX,iphone,2018-11-30 05:18:49
6,Buffer,AZ,iphone,2018-11-30 05:08:02


In [213]:
ddf = new_df.rename(columns={'location': 'school_state', 'time': 'date_posted', 'tag': 'resource_type'})

In [214]:
ddf.head(10)

,device,school_state,resource_type,date_posted
0,Andriod,FL,ipad,2018-11-30 05:44:03
1,Andriod,FL,ipad,2018-11-30 05:40:01
2,Andriod,FL,ipad,2018-11-30 05:35:58
3,Andriod,FL,ipad,2018-11-30 05:31:56
4,Andriod,FL,ipad,2018-11-30 05:26:55
5,Web,TX,iphone,2018-11-30 05:18:49
6,Buffer,AZ,iphone,2018-11-30 05:08:02
